In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.display.max_columns = 100000

In [8]:
teams = pd.read_csv('data/cbb.csv')
teams_2021 = pd.read_csv('data/cbb21.csv')
tourney_teams_2021 = teams_2021.loc[teams_2021.SEED.notnull()]
teams['SEED_str'] = teams['SEED'].astype(pd.StringDtype())
teams['Adj_OtoD_Margin'] = teams['ADJOE']-teams['ADJDE']
teams_2021['SEED_str'] = teams_2021['SEED'].astype(pd.StringDtype())
#tourney_teams_2021['SEED_str'] = tourney_teams_2021['SEED'].astype(pd.StringDtype())
tourney_teams_2021 = teams_2021.loc[teams_2021.SEED.notnull()]

In [9]:
# Data crunching to compare past champions to average of each round

tourney_teams = teams.loc[teams.SEED.notnull()]
champions = tourney_teams[tourney_teams["POSTSEASON"].str.contains('Champions')]
not_champions = tourney_teams[~tourney_teams["POSTSEASON"].str.contains('Champions')]
runner_ups = tourney_teams[tourney_teams["POSTSEASON"].str.contains('2ND')]
final4 = tourney_teams[tourney_teams["POSTSEASON"].str.contains('F4')]
elite8 = tourney_teams[tourney_teams["POSTSEASON"].str.contains('E8')]
sweet16 = tourney_teams[tourney_teams["POSTSEASON"].str.contains('S16')]
round32 = tourney_teams[tourney_teams["POSTSEASON"].str.contains('R32')]
round64 = tourney_teams[tourney_teams["POSTSEASON"].str.contains('R64')]
round68 = tourney_teams[tourney_teams["POSTSEASON"].str.contains('R68')]
tourney_teams['isChampion'] = np.where(tourney_teams["POSTSEASON"].str.contains('Champions'), 1, 0)

# AVGs for non-champions\n",
not_champs_avg_stats = not_champions.mean()
not_champs_avg_stats_df = not_champs_avg_stats.to_frame()
not_champs_avg_stats_df = not_champs_avg_stats_df.transpose()
not_champs_avg_stats_df.insert(0,"TEAM", "AVG OF NON CHAMPS")
not_champs_avg_stats_df.insert(1,"CONF", "N/A CONF")
not_champs_avg_stats_df.insert(21,"POSTSEASON", "Avg Tourney Team")
not_champs_avg_stats_df.insert(24,"SEED_str", "N/A SEED")
champ_vs_avg_df = champions.append(not_champs_avg_stats_df)

# AVGs for runner ups\n",
runner_ups_avg_stats = runner_ups.mean()
runner_ups_avg_stats_df = runner_ups_avg_stats.to_frame()
runner_ups_avg_stats_df = runner_ups_avg_stats_df.transpose()
runner_ups_avg_stats_df.insert(0,"TEAM", "Runner Ups")
runner_ups_avg_stats_df.insert(1,"CONF", "N/A CONF")
runner_ups_avg_stats_df.insert(21,"POSTSEASON", "Avg Runner Up")
runner_ups_avg_stats_df.insert(24,"SEED_str", "N/A SEED")
champ_vs_avg_df = champ_vs_avg_df.append(runner_ups_avg_stats_df)

# AVGs for final four teams
f4_avg_stats = final4.mean()
f4_avg_stats_df = f4_avg_stats.to_frame()
f4_avg_stats_df = f4_avg_stats_df.transpose()
f4_avg_stats_df.insert(0,"TEAM", "Final Four")
f4_avg_stats_df.insert(1,"CONF", "N/A CONF")
f4_avg_stats_df.insert(21,"POSTSEASON", "Avg F4 Team")
f4_avg_stats_df.insert(24,"SEED_str", "N/A SEED")
champ_vs_avg_df = champ_vs_avg_df.append(f4_avg_stats_df)

# AVGs for elite eight teams
e8_avg_stats = elite8.mean()
e8_avg_stats_df = e8_avg_stats.to_frame()
e8_avg_stats_df = e8_avg_stats_df.transpose()
e8_avg_stats_df.insert(0,"TEAM", "Elite Eight")
e8_avg_stats_df.insert(1,"CONF", "N/A CONF")
e8_avg_stats_df.insert(21,"POSTSEASON", "Avg E8 Team")
e8_avg_stats_df.insert(24,"SEED_str", "N/A SEED")
champ_vs_avg_df = champ_vs_avg_df.append(e8_avg_stats_df)

# AVGs for sweet sixteen teams
s16_avg_stats = sweet16.mean()
s16_avg_stats_df = s16_avg_stats.to_frame()
s16_avg_stats_df = s16_avg_stats_df.transpose()
s16_avg_stats_df.insert(0,"TEAM", "Sweet Sixteen")
s16_avg_stats_df.insert(1,"CONF", "N/A CONF")
s16_avg_stats_df.insert(21,"POSTSEASON", "Avg S16 Team")
s16_avg_stats_df.insert(24,"SEED_str", "N/A SEED")
champ_vs_avg_df = champ_vs_avg_df.append(s16_avg_stats_df)

# AVGs for round of 32 teams
r32_avg_stats = round32.mean()
r32_avg_stats_df = r32_avg_stats.to_frame()
r32_avg_stats_df = r32_avg_stats_df.transpose()
r32_avg_stats_df.insert(0,"TEAM", "Round of 32")
r32_avg_stats_df.insert(1,"CONF", "N/A CONF")
r32_avg_stats_df.insert(21,"POSTSEASON", "Avg R32 Team")
r32_avg_stats_df.insert(24,"SEED_str", "N/A SEED"),
champ_vs_avg_df = champ_vs_avg_df.append(r32_avg_stats_df)

# AVGs for round of 64 teams
r64_avg_stats = round64.mean()
r64_avg_stats_df = r64_avg_stats.to_frame()
r64_avg_stats_df = r64_avg_stats_df.transpose()
r64_avg_stats_df.insert(0,"TEAM", "Round of 64")
r64_avg_stats_df.insert(1,"CONF", "N/A CONF")
r64_avg_stats_df.insert(21,"POSTSEASON", "Avg R64 Team")
r64_avg_stats_df.insert(24,"SEED_str", "N/A SEED")
champ_vs_avg_df = champ_vs_avg_df.append(r64_avg_stats_df)

# AVGs for round of 68 teams
r68_avg_stats = round68.mean()
r68_avg_stats_df = r68_avg_stats.to_frame()
r68_avg_stats_df = r68_avg_stats_df.transpose()
r68_avg_stats_df.insert(0,"TEAM", "Round of 68")
r68_avg_stats_df.insert(1,"CONF", "N/A CONF")
r68_avg_stats_df.insert(21,"POSTSEASON", "Avg R68 Team")
r68_avg_stats_df.insert(24,"SEED_str", "N/A SEED")
champ_vs_avg_df = champ_vs_avg_df.append(r68_avg_stats_df)

C:\Users\mattg\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [3]:
tourney_teams_2023 = pd.read_csv('data/2023_team_results.csv')
tourney_teams_2023['Adj_OtoD_Margin'] = tourney_teams_2023['adjoe']-tourney_teams_2023['adjde']
teams_2023_final = pd.read_csv('data/2023_fffinal.csv')
teams_2023_final = teams_2023_final.rename(columns={"TeamName": "team"})
tourney_teams_2023_join = tourney_teams_2023.merge(teams_2023_final, on="team")
championship_candidates = tourney_teams_2023_join[(tourney_teams_2023_join["Adj_OtoD_Margin"] > 25) & (tourney_teams_2023_join["TO%"] < 18)]
championship_candidates.sort_values("Adj_OtoD_Margin", ascending=False)

,rank,team,conf,record,adjoe,oe Rank,adjde,de Rank,barthag,rank.1,proj. W,Proj. L,Pro Con W,Pro Con L,Con Rec.,sos,ncsos,consos,Proj. SOS,Proj. Noncon SOS,Proj. Con SOS,elite SOS,elite noncon SOS,Opp OE,Opp DE,Opp Proj. OE,Opp Proj DE,Con Adj OE,Con Adj DE,Qual O,Qual D,Qual Barthag,Qual Games,FUN,ConPF,ConPA,ConPoss,ConOE,ConDE,ConSOSRemain,Conf Win%,WAB,WAB Rk,"Fun Rk, adjt",Adj_OtoD_Margin,eFG%,eFG% Def,FTR,FTR Def,OR%,DR%,TO%,TO% Def.,3P%,3pD%,2p%,2p%D,ft%,ft%D,3P rate,3P rate D,arate,arateD
0,1,Houston,Amer,31-3,118.260139,12,89.970231,4,0.958679,1,31.955017,3.044983,17,1,17-1,0.616414,0.537336,0.654527,0.613721,0.537336,0.654527,0.768128,0.784425,107.504127,101.943296,107.378687,101.954088,122.010074,94.505982,123.465231,92.824828,0.963748,10,0.035108,1392,1104,1164.8625,1.194991,0.947751,0,0.944444,8.395558,5,106,28.289908,52.7,42.5,28.5,35.8,37.1,27.4,15.3,22.0,34.5,27.8,53.2,43.1,71.9,69.0,37.8,43.7,53.8,53.6
6,7,Gonzaga,WCC,28-5,125.199540,1,99.674033,80,0.932267,7,28.889635,5.110365,14,2,14-2,0.643860,0.612710,0.652126,0.643472,0.612710,0.652126,0.737805,0.718299,110.102984,101.838434,110.159501,101.991155,125.777731,101.370918,137.113806,105.506057,0.953180,10,0.056001,1431,1216,1153.7125,1.240344,1.053989,0,0.875000,7.074019,9,62,25.525507,58.5,51.7,34.2,26.7,31.1,26.0,14.6,18.8,38.4,35.0,59.0,51.2,70.2,73.5,32.8,39.4,50.7,45.3
5,6,Purdue,B10,29-5,119.494854,7,94.247513,21,0.938746,6,29.000000,5.000000,15,5,15-5,0.705650,0.544207,0.775853,0.705650,0.544207,0.775853,0.704254,0.780309,109.834133,100.430317,109.834133,100.430317,118.184136,93.928855,130.685688,102.677415,0.941245,15,0.062391,1418,1282,1274.2500,1.112811,1.006082,0,0.750000,9.178229,3,51,25.247341,52.2,47.2,37.2,19.3,38.3,23.3,17.0,15.4,32.6,31.4,54.1,47.4,74.3,73.6,38.1,35.4,60.9,51.5
2,3,UCLA,P12,29-5,113.602138,33,88.530852,2,0.946218,3,29.948633,5.051367,18,2,18-2,0.687179,0.531127,0.748903,0.681485,0.531127,0.748903,0.724026,0.805413,109.129526,100.286733,108.943759,100.366009,112.249097,87.574005,121.887739,96.794755,0.934063,14,0.034992,1400,1186,1311.5500,1.067439,0.904274,0,0.900000,8.439189,4,107,25.071286,50.9,46.8,27.1,27.6,33.7,27.6,15.3,23.6,34.7,31.1,50.4,46.8,72.2,69.7,29.1,39.3,50.9,51.3


In [14]:
tourney_teams_2023

,rank,team,conf,record,adjoe,oe Rank,adjde,de Rank,barthag,rank.1,proj. W,Proj. L,Pro Con W,Pro Con L,Con Rec.,sos,ncsos,consos,Proj. SOS,Proj. Noncon SOS,Proj. Con SOS,elite SOS,elite noncon SOS,Opp OE,Opp DE,Opp Proj. OE,Opp Proj DE,Con Adj OE,Con Adj DE,Qual O,Qual D,Qual Barthag,Qual Games,FUN,ConPF,ConPA,ConPoss,ConOE,ConDE,ConSOSRemain,Conf Win%,WAB,WAB Rk,"Fun Rk, adjt",Adj_OtoD_Margin
1,Houston,Amer,31-3,118.260139,12,89.970231,4,0.958679,1,31.955017,3.044983,17,1,17-1,0.616414,0.537336,0.654527,0.613721,0.537336,0.654527,0.768128,0.784425,107.504127,101.943296,107.378687,101.954088,122.010074,94.505982,123.465231,92.824828,0.963748,10,0.035108,1392.0,1104.0,1164.8625,1.194991,0.947751,0,0.944444,8.395558,5,106,63.262210,8
2,Alabama,SEC,29-5,116.980841,15,89.393018,3,0.956608,2,29.000000,5.000000,16,2,16-2,0.740135,0.702063,0.751423,0.740135,0.702063,0.751423,0.661333,0.661623,110.605759,98.798200,110.605759,98.798200,117.972148,87.721540,117.874992,94.475242,0.927223,17,0.051723,1495.0,1198.0,1321.8250,1.131012,0.906323,0,0.888889,10.551228,1,70,72.703653,12
3,UCLA,P12,29-5,113.602139,33,88.530852,2,0.946218,3,29.948633,5.051367,18,2,18-2,0.687179,0.531127,0.748903,0.681485,0.531127,0.748903,0.724026,0.805413,109.129526,100.286733,108.943759,100.366009,112.249097,87.574005,121.887739,96.794755,0.934063,14,0.034992,1400.0,1186.0,1311.5500,1.067439,0.904274,0,0.900000,8.439189,4,107,66.345234,31
4,Tennessee,SEC,23-10,112.386025,47,88.220245,1,0.941815,4,23.853093,10.146907,11,7,11-7,0.677147,0.568079,0.750442,0.678877,0.568079,0.750442,0.715705,0.761803,108.944450,100.425161,109.047093,100.496350,112.415079,88.734646,116.882163,92.019462,0.939937,13,-0.104783,1250.0,1077.0,1163.4125,1.074425,0.925725,0,0.611111,3.124925,25,351,65.582471,46
5,Connecticut,BE,25-8,119.813209,6,94.128383,20,0.941291,5,25.818120,8.181880,13,7,13-7,0.664885,0.502880,0.734646,0.668298,0.502880,0.734646,0.716468,0.787490,108.127506,100.573539,108.182491,100.512193,117.157999,95.002274,124.326509,97.443387,0.942776,13,-0.043554,1513.0,1367.0,1355.6500,1.116070,1.008372,0,0.650000,4.968453,14,276,66.980217,-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,Delaware St.,MEAC,6-23,91.619226,356,111.636866,319,0.093425,359,7.000000,22.000000,4,10,4-10,0.376478,0.454884,0.289759,0.376478,0.454884,0.289759,0.868691,0.788911,101.915226,105.624696,101.915226,105.624696,92.637165,112.208930,87.839734,93.121099,0.338173,4,-0.053654,938.0,1055.0,990.2250,0.947259,1.065414,0,0.285714,-17.511344,343,297,67.680170,37
360,IUPUI,Horz,5-27,96.158581,330,118.422602,360,0.083555,360,5.000000,27.000000,2,18,2-18,0.350852,0.298412,0.376956,0.350852,0.298412,0.376956,0.906959,0.878243,102.312337,107.724320,102.312337,107.724320,95.995288,118.330007,73.093119,104.709037,0.015771,2,-0.080168,1350.0,1599.0,1363.1250,0.990371,1.173040,0,0.100000,-22.327374,361,334,67.128851,-30
361,Green Bay,Horz,3-29,91.186360,358,114.920738,349,0.065352,361,3.000000,29.000000,2,18,2-18,0.437154,0.535298,0.378824,0.437154,0.535298,0.378824,0.884176,0.816909,103.792781,106.600318,103.792781,106.600318,91.088556,115.059997,0.000000,0.000000,0.000000,0,-0.027467,1227.0,1538.0,1330.5000,0.922210,1.155956,0,0.100000,-22.994167,363,235,65.226955,9
362,Hartford,ind,5-23,90.226486,361,118.470256,361,0.041811,362,5.000000,23.000000,0,2,0-2,0.281556,0.285154,0.234780,0.281556,0.285154,0.234780,0.933329,0.931087,99.963502,108.287082,99.963502,108.287082,80.075411,110.960943,0.000000,0.000000,0.000000,0,-0.060201,102.0,137.0,123.1125,0.828511,1.112803,0,0.000000,-19.933252,357,302,65.531662,0


In [11]:
tourney_teams_2023_join.head(5)

,rank,team,conf,record,adjoe,oe Rank,adjde,de Rank,barthag,rank.1,proj. W,Proj. L,Pro Con W,Pro Con L,Con Rec.,sos,ncsos,consos,Proj. SOS,Proj. Noncon SOS,Proj. Con SOS,elite SOS,elite noncon SOS,Opp OE,Opp DE,Opp Proj. OE,Opp Proj DE,Con Adj OE,Con Adj DE,Qual O,Qual D,Qual Barthag,Qual Games,FUN,ConPF,ConPA,ConPoss,ConOE,ConDE,ConSOSRemain,Conf Win%,WAB,WAB Rk,"Fun Rk, adjt",Fun adjt,Adj_OtoD_Margin,eFG%,eFG% Def,FTR,FTR Def,OR%,DR%,TO%,TO% Def.,3P%,3pD%,2p%,2p%D,ft%,ft%D,3P rate,3P rate D,arate,arateD
0,1,Gonzaga,WCC,26-3,121.393077,3,88.462846,4,0.974399,1,26.965166,3.034834,13,1,13-1,0.583557,0.492788,0.623477,0.583395,0.492788,0.623477,0.752212,0.777635,104.362214,99.575644,104.243616,99.525107,120.872315,88.183632,138.914015,98.487598,0.981204,12,-0.007154,1271,942,1063.5500,1.195054,0.885713,0,0.928571,7.017371,8,192,72.798387,32.930231,59.4,43.2,29.7,22.2,29.0,23.0,15.9,17.0,37.9,30.7,60.9,41.6,73.1,69.5,35.4,33.9,55.0,40.6
1,2,Houston,Amer,29-5,116.970661,10,89.185771,7,0.957662,2,29.846047,5.153953,15,3,15-3,0.643936,0.562090,0.678483,0.648525,0.562090,0.678483,0.771671,0.819246,104.797261,98.680693,104.989664,98.678953,116.467094,91.355916,114.370176,97.264820,0.865650,8,-0.032147,1320,1099,1188.0375,1.111076,0.925055,0,0.833333,6.451982,12,244,64.027806,27.784889,53.6,43.5,28.7,35.1,37.8,27.3,17.1,21.7,34.1,29.0,55.1,43.5,66.9,70.5,39.1,42.9,59.5,55.4
2,3,Kansas,B12,28-6,119.854423,5,93.094649,24,0.948123,3,28.956748,6.043252,14,4,14-4,0.769667,0.628368,0.857039,0.760603,0.628368,0.857039,0.657761,0.793914,107.349160,94.803196,107.049472,94.908286,119.445183,93.929611,123.063636,97.046808,0.938851,19,0.053230,1352,1244,1254.3000,1.077892,0.991788,0,0.777778,10.286754,1,65,69.120781,26.759774,54.1,46.9,32.8,27.8,33.4,28.9,17.8,18.4,35.5,30.1,54.5,47.9,72.4,70.7,33.8,34.2,53.9,47.5
3,4,Baylor,B12,26-6,117.065860,9,91.612885,17,0.943715,4,26.949811,6.050189,14,4,14-4,0.716506,0.510003,0.857544,0.709029,0.510003,0.857544,0.665500,0.800306,106.544444,96.658470,106.382523,96.828580,117.627166,92.374775,123.871013,96.701682,0.945190,18,0.051949,1298,1195,1220.6375,1.063379,0.978997,0,0.777778,8.826070,3,69,67.545202,25.452975,52.9,47.8,28.5,26.9,36.3,28.4,18.2,22.9,34.6,29.9,53.5,49.5,69.8,70.1,37.7,35.9,56.1,55.5
4,5,Texas Tech,B12,25-9,108.799044,59,85.348325,1,0.942229,5,25.926097,9.073903,12,6,6-Dec,0.681915,0.393592,0.857714,0.678589,0.393592,0.857714,0.662968,0.823048,106.399451,97.260564,106.318009,97.373205,109.442047,86.984964,111.548773,88.045587,0.938253,19,-0.013135,1231,1133,1221.2625,1.007973,0.927728,0,0.666667,6.376356,13,207,66.373105,23.450718,52.4,45.5,36.1,32.1,33.3,26.1,20.3,23.6,31.4,31.3,55.1,44.3,69.9,68.3,34.4,45.6,52.3,53.8


In [16]:
#################################
#### MARCH MADNESS SIMULATOR ####
#################################

# Functions for running tournament simulator\n",
# Needs: High ADJOE and Low ADJDE (take care of this with margin), Low TOR\n",
# Wants: Low FTRD, High EFG, 3P_D, 2P_O\n",
# Other Notes...\n",
# Needs: ADJOE-ADJDE > 30, TOR < 16.5\n",
# Nice to have: FTRD < 31\n",
# Marg(0.61)up, TOR(0.14)*1.2down, Low FTRD(0.11)*.85down, Low 3P_D(0.08)down, High 2P_O(0.06)*0.5up\n",

def game_predictor(df, team1, team2):
    # Team 1 stats
    team1_df = df[df["team"].str.contains(team1)]
    team1_o2d_marg = team1_df["Adj_OtoD_Margin"]
    team1_TOR = team1_df["TO%"]
    team1_FTRD = team1_df["FTR Def"]
    team1_3PD = team1_df["3pD%"]
    team1_2PO = team1_df["2p%"]

    # Team 2 stats\n",
    team2_df = df[df["team"].str.contains(team2)]
    team2_o2d_marg = team2_df["Adj_OtoD_Margin"]
    team2_TOR = team2_df["TO%"]
    team2_FTRD = team2_df["FTR Def"]
    team2_3PD = team2_df["3pD%"]
    team2_2PO = team2_df["2p%"]

    #team1_total = team1_o2d_marg - (team1_TOR * 0.69)  #Initial equation
    team1_total = (team1_o2d_marg*0.61) - (team1_TOR*0.14*1.2) - (team1_FTRD*0.11*0.85) - (team1_3PD*0.08) + (team1_2PO*0.06*0.5)
    team1_total = team1_total.to_numpy()[0]
    #print(team1_total)
    #print(str(team1)+\" Total: \" + str(round(team1_total, 2)))
    #team2_total = team2_o2d_marg - (team2_TOR * 0.69)  #Initial equation
    team2_total = (team2_o2d_marg*0.61) - (team2_TOR*0.14*1.2) - (team2_FTRD*0.11*0.85) - (team2_3PD*0.08) + (team2_2PO*0.06*0.5)
    team2_total = team2_total.to_numpy()[0]
    #print(str(team2)+\" Total: \" + str(round(team2_total, 2)))

    result = team1_total > team2_total
    margin_of_pred = abs(team1_total - team2_total)
    if result == True:
        print("Winner: " + str(team1))
        return team1, margin_of_pred
    else:
        print("Winner: " + str(team2))
        return team2, margin_of_pred

def play_round(teams_remaining, stats_df):
    num_games = len(teams_remaining)//2
    print(num_games)
    winners_list = []
    margin_of_pred_list = []
    for game in range(0,num_games):
        team1_index = game*2
        team2_index = game*2+1
        team1_name = teams_remaining[team1_index]
        print("")
        print(team1_name)
        team2_name = teams_remaining[team2_index]
        print(team2_name)
        winner, margin_of_pred = game_predictor(stats_df,team1_name,team2_name)
        winners_list.append(winner)
        margin_of_pred_list.append(margin_of_pred)
    return winners_list, margin_of_pred_list

In [17]:
#Testing 1 on 1 game prediction
game_predictor(tourney_teams_2023_join,"Duke","Oral Roberts")

Winner: Duke


('Duke', 3.0872551672000053)

In [10]:
#Initialize Year
year = 2023

# Initialize Teams
df_round68 = ["Texas A&M Corpus Chris","Southeastern Louisiana","Texas Southern","Fairleigh Dickinson","Mississippi St.","Pittsburgh","Arizona St.","Nevada"]
df_round64 = ['Alabama','Texas A&M Corpus Chris','Maryland','West Virginia','San Diego St.','College of Charleston','Virginia','Furman',
              'Creighton','North Carolina St.','Baylor','UC Santa Barbara','Missouri','Utah St.','Arizona','Princeton',
              'Purdue','Texas Southern','Memphis','Florida Atlantic',"Duke",'Oral Roberts','Tennessee','Louisiana Lafayette','Kentucky','Providence',
              'Kansas St.','Montana St.','Michigan St.','USC','Marquette','Vermont',
              'Houston','Northern Kentucky','Iowa','Auburn','Miami FL','Drake','Indiana','Kent St.','Iowa St.','Mississippi St.',
              'Xavier','Kennesaw St.','Texas A&M','Penn St.','Texas','Colgate',
              'Kansas','Howard','Arkansas','Illinois',"Saint Mary's",'VCU','Connecticut','Iona','TCU','Nevada',
              'Gonzaga','Grand Canyon','Northwestern','Boise St.','UCLA','UNC Asheville']
df_round32 = []
df_round16 = []
df_round8 = []
df_round4 = []
df_round2 = []
df_round1 = []
margins64 = ["None here b/c no games played yet, go to next round"]
margins32 = []
margins16 = []
margins8 = []
margins4 = []
margins2 = []
margins1 = []

# Combine list of rounds
list_of_rounds = [df_round64,df_round32,df_round16,df_round8,df_round4,df_round2,df_round1]
pred_win_margins = [margins64,margins32,margins16,margins8,margins4,margins2,margins1]

# Play the tournament
print("This is March! Let the tournament begin...")
inv_round_counter = 5
for round in range(0,6):
    remaining_teams = 2**inv_round_counter
    winners, margins = play_round(list_of_rounds[round],tourney_teams_2023_join)
    list_of_rounds[round+1] = winners
    winners_df = pd.DataFrame(winners)
    #winners_df = winners_df.rename(index={0: "Remaining Teams"})
    win_csv_name = "sim_output/"+str(year)+"/roundof"+str(remaining_teams)+"_winners.csv"
    winners_df.to_csv(win_csv_name)
    pred_win_margins[round] = margins
    margins_df = pd.DataFrame(margins)
    #margins_df = margins_df.rename(index={0: "Predictor Score Margin of Victory"})
    margin_csv_name = "sim_output/"+str(year)+"/roundof"+str(remaining_teams)+"_margins.csv"
    margins_df.to_csv(margin_csv_name)
    inv_round_counter -= 1
champion = list_of_rounds[6][0]
print("")
print("")
print("***Congrats, your predicted champion is "+str(champion)+"!***")
print("For the results of all the rounds, check your file directory for each game's winner, or use the two cells below for quick access.")

This is March! Let the tournament begin...
32

Alabama
Texas A&M Corpus Chris
Winner: Alabama

Maryland
West Virginia
Winner: West Virginia

San Diego St.
College of Charleston
Winner: San Diego St.

Virginia
Furman
Winner: Virginia

Creighton
North Carolina St.
Winner: Creighton

Baylor
UC Santa Barbara
Winner: Baylor

Missouri
Utah St.
Winner: Utah St.

Arizona
Princeton
Winner: Arizona

Purdue
Texas Southern
Winner: Purdue

Memphis
Florida Atlantic
Winner: Memphis

Duke
Oral Roberts
Winner: Duke

Tennessee
Louisiana Lafayette
Winner: Tennessee

Kentucky
Providence
Winner: Kentucky

Kansas St.
Montana St.
Winner: Kansas St.

Michigan St.
USC
Winner: Michigan St.

Marquette
Vermont
Winner: Marquette

Houston
Northern Kentucky
Winner: Houston

Iowa
Auburn
Winner: Auburn

Miami FL
Drake
Winner: Miami FL

Indiana
Kent St.
Winner: Indiana

Iowa St.
Mississippi St.
Winner: Iowa St.

Xavier
Kennesaw St.
Winner: Xavier

Texas A&M
Penn St.
Winner: Texas A&M

Texas
Colgate
Winner: Texas

Kansa

In [22]:
# Run cell to see the winners for a chosen round

list_of_rounds[1]

['Alabama',
 'West Virginia',
 'San Diego St.',
 'Virginia',
 'Creighton',
 'Baylor',
 'Utah St.',
 'Arizona',
 'Purdue',
 'Memphis',
 'Duke',
 'Tennessee',
 'Kentucky',
 'Kansas St.',
 'Michigan St.',
 'Marquette',
 'Houston',
 'Auburn',
 'Miami FL',
 'Indiana',
 'Iowa St.',
 'Xavier',
 'Texas A&M',
 'Texas',
 'Kansas',
 'Arkansas',
 "Saint Mary's",
 'Connecticut',
 'TCU',
 'Gonzaga',
 'Boise St.',
 'UCLA']

In [19]:
# Run cell to see the simulators win margins for a chosen round

pred_win_margins[3]

[1.8209546108999994,
 2.8132028242000064,
 3.3861182791000006,
 1.5152153489999964]

# March Madness Simulator

In [68]:
tourney_teams_2023 = pd.read_csv('data/2023_team_results.csv')
tourney_teams_2023['Adj_OtoD_Margin'] = tourney_teams_2023['adjoe']-tourney_teams_2023['adjde']
teams_2023_final = pd.read_csv('data/2023_fffinal.csv')
teams_2023_final = teams_2023_final.rename(columns={"TeamName": "team"})
tourney_teams_2023_join = tourney_teams_2023.merge(teams_2023_final, on="team")
championship_candidates = tourney_teams_2023_join[(tourney_teams_2023_join["Adj_OtoD_Margin"] > 25) & (tourney_teams_2023_join["TO%"] < 18)]
championship_candidates.sort_values("Adj_OtoD_Margin", ascending=False)

,rank,team,conf,record,adjoe,oe Rank,adjde,de Rank,barthag,rank.1,proj. W,Proj. L,Pro Con W,Pro Con L,Con Rec.,sos,ncsos,consos,Proj. SOS,Proj. Noncon SOS,Proj. Con SOS,elite SOS,elite noncon SOS,Opp OE,Opp DE,Opp Proj. OE,Opp Proj DE,Con Adj OE,Con Adj DE,Qual O,Qual D,Qual Barthag,Qual Games,FUN,ConPF,ConPA,ConPoss,ConOE,ConDE,ConSOSRemain,Conf Win%,WAB,WAB Rk,"Fun Rk, adjt",Adj_OtoD_Margin,eFG%,eFG% Def,FTR,FTR Def,OR%,DR%,TO%,TO% Def.,3P%,3pD%,2p%,2p%D,ft%,ft%D,3P rate,3P rate D,arate,arateD
0,1,Houston,Amer,31-3,118.260139,12,89.970231,4,0.958679,1,31.955017,3.044983,17,1,17-1,0.616414,0.537336,0.654527,0.613721,0.537336,0.654527,0.768128,0.784425,107.504127,101.943296,107.378687,101.954088,122.010074,94.505982,123.465231,92.824828,0.963748,10,0.035108,1392,1104,1164.8625,1.194991,0.947751,0,0.944444,8.395558,5,106,28.289908,52.7,42.5,28.5,35.8,37.1,27.4,15.3,22.0,34.5,27.8,53.2,43.1,71.9,69.0,37.8,43.7,53.8,53.6
6,7,Gonzaga,WCC,28-5,125.199540,1,99.674033,80,0.932267,7,28.889635,5.110365,14,2,14-2,0.643860,0.612710,0.652126,0.643472,0.612710,0.652126,0.737805,0.718299,110.102984,101.838434,110.159501,101.991155,125.777731,101.370918,137.113806,105.506057,0.953180,10,0.056001,1431,1216,1153.7125,1.240344,1.053989,0,0.875000,7.074019,9,62,25.525507,58.5,51.7,34.2,26.7,31.1,26.0,14.6,18.8,38.4,35.0,59.0,51.2,70.2,73.5,32.8,39.4,50.7,45.3
5,6,Purdue,B10,29-5,119.494854,7,94.247513,21,0.938746,6,29.000000,5.000000,15,5,15-5,0.705650,0.544207,0.775853,0.705650,0.544207,0.775853,0.704254,0.780309,109.834133,100.430317,109.834133,100.430317,118.184136,93.928855,130.685688,102.677415,0.941245,15,0.062391,1418,1282,1274.2500,1.112811,1.006082,0,0.750000,9.178229,3,51,25.247341,52.2,47.2,37.2,19.3,38.3,23.3,17.0,15.4,32.6,31.4,54.1,47.4,74.3,73.6,38.1,35.4,60.9,51.5
2,3,UCLA,P12,29-5,113.602138,33,88.530852,2,0.946218,3,29.948633,5.051367,18,2,18-2,0.687179,0.531127,0.748903,0.681485,0.531127,0.748903,0.724026,0.805413,109.129526,100.286733,108.943759,100.366009,112.249097,87.574005,121.887739,96.794755,0.934063,14,0.034992,1400,1186,1311.5500,1.067439,0.904274,0,0.900000,8.439189,4,107,25.071286,50.9,46.8,27.1,27.6,33.7,27.6,15.3,23.6,34.7,31.1,50.4,46.8,72.2,69.7,29.1,39.3,50.9,51.3


In [69]:
# Functions for running tournament simulator\n",
# Needs: High ADJOE and Low ADJDE (take care of this with margin), Low TOR\n",
# Wants: Low FTRD, High EFG, 3P_D, 2P_O\n",
# Other Notes...\n",
# Needs: ADJOE-ADJDE > 30, TOR < 16.5\n",
# Nice to have: FTRD < 31\n",
# Marg(0.61)up, TOR(0.14)*1.2down, Low FTRD(0.11)*.85down, Low 3P_D(0.08)down, High 2P_O(0.06)*0.5up\n",

def stat_scaler(df, stat, inverse_stat=False):
    stat_min = df[stat].min()
    stat_max = df[stat].max()
    if inverse_stat == False:
        scaled_data = (df[stat]-stat_min)/(stat_max-stat_min) # Normalized stat (0-1 Min-Max Scaler)
        return scaled_data
    else: 
        scaled_data = 1-((df[stat]-stat_min)/(stat_max-stat_min))  # Inversed Normalized stat (0-1 Max Min Scaler)
        return scaled_data


def multi_stat_scaler(df, stat_weights, inverse_stat_flags):
    for stat in stat_weights.columns:
        inverse_flag = inverse_stat_flags[stat][0]
        if inverse_flag == False:
            df[stat] = stat_scaler(df, stat)
        else:
            df[stat] = stat_scaler(df, stat, inverse_stat=True)
    return df
    

def team_power_rating_generator(scaled_df, team, stat_weights):
    team_df = scaled_df[scaled_df["team"].str.contains(team)]
    team_pr = 0
    for stat in stat_weights:
        team_pr += team_df[stat].iloc[0] * stat_weights[stat].iloc[0]
    return team_pr


def game_predictor(scaled_df, stat_weights, team1, team2):
    # Team 1 Power Rating
    team1_pr = team_power_rating_generator(scaled_df, team1, stat_weights)

    # Team 2 Power Rating
    team2_pr = team_power_rating_generator(scaled_df, team2, stat_weights)

    result = team1_pr > team2_pr
    margin_of_pred = abs(team1_pr - team2_pr)
    if result == True:
        print("Winner: " + str(team1))
        return team1, margin_of_pred
    else:
        print("Winner: " + str(team2))
        return team2, margin_of_pred

def play_round(teams_remaining, stats_df, stat_weights):
    num_games = len(teams_remaining)//2
    print(num_games)
    winners_list = []
    margin_of_pred_list = []
    for game in range(0,num_games):
        team1_index = game*2
        team2_index = game*2+1
        team1_name = teams_remaining[team1_index]
        print("")
        print(team1_name)
        team2_name = teams_remaining[team2_index]
        print(team2_name)
        winner, margin_of_pred = game_predictor(stats_df,stat_weights,team1_name,team2_name)
        winners_list.append(winner)
        margin_of_pred_list.append(margin_of_pred)
    return winners_list, margin_of_pred_list

In [70]:
# Initialize Year
year = 2023

# User
username = "original"

# Scale Tournament Data
stat_weights = pd.read_csv('model_weights/'+username+'_model_weights.csv')
inverse_stat_flags = pd.read_csv('model_weights/inverse_stat_flags.csv')
scaled_stat_df = multi_stat_scaler(tourney_teams_2023_join, stat_weights, inverse_stat_flags)


# Initialize Teams
df_round68 = ["Texas A&M Corpus Chris","Southeastern Louisiana","Texas Southern","Fairleigh Dickinson","Mississippi St.","Pittsburgh","Arizona St.","Nevada"]
df_round64 = ['Alabama','Texas A&M Corpus Chris','Maryland','West Virginia','San Diego St.','College of Charleston','Virginia','Furman',
              'Creighton','North Carolina St.','Baylor','UC Santa Barbara','Missouri','Utah St.','Arizona','Princeton',
              'Purdue','Texas Southern','Memphis','Florida Atlantic',"Duke",'Oral Roberts','Tennessee','Louisiana Lafayette','Kentucky','Providence',
              'Kansas St.','Montana St.','Michigan St.','USC','Marquette','Vermont',
              'Houston','Northern Kentucky','Iowa','Auburn','Miami FL','Drake','Indiana','Kent St.','Iowa St.','Mississippi St.',
              'Xavier','Kennesaw St.','Texas A&M','Penn St.','Texas','Colgate',
              'Kansas','Howard','Arkansas','Illinois',"Saint Mary's",'VCU','Connecticut','Iona','TCU','Nevada',
              'Gonzaga','Grand Canyon','Northwestern','Boise St.','UCLA','UNC Asheville']
df_round32 = []
df_round16 = []
df_round8 = []
df_round4 = []
df_round2 = []
df_round1 = []
margins64 = ["None here b/c no games played yet, go to next round"]
margins32 = []
margins16 = []
margins8 = []
margins4 = []
margins2 = []
margins1 = []

# Combine list of rounds
list_of_rounds = [df_round64,df_round32,df_round16,df_round8,df_round4,df_round2,df_round1]
pred_win_margins = [margins64,margins32,margins16,margins8,margins4,margins2,margins1]

# Play the tournament
print("This is March! Let the tournament begin...")
inv_round_counter = 5
for round in range(0,6):
    remaining_teams = 2**inv_round_counter
    winners, margins = play_round(list_of_rounds[round],scaled_stat_df, stat_weights)
    list_of_rounds[round+1] = winners
    winners_df = pd.DataFrame(winners)
    #winners_df = winners_df.rename(index={0: "Remaining Teams"})
    win_csv_name = "sim_output/"+str(year)+"/"+str(username)+"/roundof"+str(remaining_teams)+"_winners.csv"
    winners_df.to_csv(win_csv_name)
    pred_win_margins[round] = margins
    margins_df = pd.DataFrame(margins)
    #margins_df = margins_df.rename(index={0: "Predictor Score Margin of Victory"})
    margin_csv_name = "sim_output/"+str(year)+"/"+str(username)+"/roundof"+str(remaining_teams)+"_margins.csv"
    margins_df.to_csv(margin_csv_name)
    inv_round_counter -= 1
champion = list_of_rounds[6][0]
print("")
print("")
print("***Congrats, your predicted champion is "+str(champion)+"!***")
print("For the results of all the rounds, check your file directory for each game's winner, or use the two cells below for quick access.")


This is March! Let the tournament begin...
32

Alabama
Texas A&M Corpus Chris
Winner: Alabama

Maryland
West Virginia
Winner: Maryland

San Diego St.
College of Charleston
Winner: San Diego St.

Virginia
Furman
Winner: Virginia

Creighton
North Carolina St.
Winner: Creighton

Baylor
UC Santa Barbara
Winner: Baylor

Missouri
Utah St.
Winner: Utah St.

Arizona
Princeton
Winner: Arizona

Purdue
Texas Southern
Winner: Purdue

Memphis
Florida Atlantic
Winner: Florida Atlantic

Duke
Oral Roberts
Winner: Oral Roberts

Tennessee
Louisiana Lafayette
Winner: Tennessee

Kentucky
Providence
Winner: Kentucky

Kansas St.
Montana St.
Winner: Kansas St.

Michigan St.
USC
Winner: Michigan St.

Marquette
Vermont
Winner: Marquette

Houston
Northern Kentucky
Winner: Houston

Iowa
Auburn
Winner: Auburn

Miami FL
Drake
Winner: Miami FL

Indiana
Kent St.
Winner: Indiana

Iowa St.
Mississippi St.
Winner: Iowa St.

Xavier
Kennesaw St.
Winner: Xavier

Texas A&M
Penn St.
Winner: Penn St.

Texas
Colgate
Winner: T

In [51]:
scaled_stat_df

,rank,team,conf,record,adjoe,oe Rank,adjde,de Rank,barthag,rank.1,proj. W,Proj. L,Pro Con W,Pro Con L,Con Rec.,sos,ncsos,consos,Proj. SOS,Proj. Noncon SOS,Proj. Con SOS,elite SOS,elite noncon SOS,Opp OE,Opp DE,Opp Proj. OE,Opp Proj DE,Con Adj OE,Con Adj DE,Qual O,Qual D,Qual Barthag,Qual Games,FUN,ConPF,ConPA,ConPoss,ConOE,ConDE,ConSOSRemain,Conf Win%,WAB,WAB Rk,"Fun Rk, adjt",Adj_OtoD_Margin,eFG%,eFG% Def,FTR,FTR Def,OR%,DR%,TO%,TO% Def.,3P%,3pD%,2p%,2p%D,ft%,ft%D,3P rate,3P rate D,arate,arateD
0,1,Houston,Amer,31-3,0.819168,12,0.054551,4,0.958679,1,31.955017,3.044983,17,1,17-1,0.616414,0.537336,0.654527,0.613721,0.537336,0.654527,0.768128,0.784425,107.504127,101.943296,107.378687,101.954088,0.917561,0.219928,123.465231,92.824828,0.963748,10,0.035108,1392,1104,1164.8625,1.194991,0.947751,0,0.944444,0.935739,5,106,1.000000,0.638037,0.059880,0.277778,0.514019,0.898058,0.575581,0.194444,0.705882,0.552448,0.891892,0.684492,0.891429,0.495050,0.670968,37.8,43.7,53.8,53.6
1,2,Alabama,SEC,29-5,0.785831,15,0.036558,3,0.956608,2,29.000000,5.000000,16,2,16-2,0.740135,0.702063,0.751423,0.740135,0.702063,0.751423,0.661333,0.661623,110.605759,98.798200,110.605759,98.798200,0.829208,0.004681,117.874992,94.475242,0.927223,17,0.051723,1495,1198,1321.8250,1.131012,0.906323,0,0.888889,1.000000,1,70,0.988073,0.638037,0.000000,0.581481,0.420561,0.737864,0.563953,0.537037,0.264706,0.503497,0.871622,0.759358,1.000000,0.529703,0.516129,47.8,30.6,54.8,41.8
2,3,UCLA,P12,29-5,0.697787,33,0.009682,2,0.946218,3,29.948633,5.051367,18,2,18-2,0.687179,0.531127,0.748903,0.681485,0.531127,0.748903,0.724026,0.805413,109.129526,100.286733,108.943759,100.366009,0.703984,0.000000,121.887739,96.794755,0.934063,14,0.034992,1400,1186,1311.5500,1.067439,0.904274,0,0.900000,0.937039,4,107,0.945321,0.527607,0.317365,0.225926,0.258567,0.733010,0.563953,0.194444,0.823529,0.566434,0.668919,0.534759,0.680000,0.509901,0.625806,29.1,39.3,50.9,51.3
3,4,Tennessee,SEC,23-10,0.666096,47,0.000000,1,0.941815,4,23.853093,10.146907,11,7,7-Nov,0.677147,0.568079,0.750442,0.678877,0.568079,0.750442,0.715705,0.761803,108.944450,100.425161,109.047093,100.496350,0.707615,0.036823,116.882164,92.019462,0.939937,13,-0.104783,1250,1077,1163.4125,1.074425,0.925725,0,0.611111,0.778619,25,351,0.929938,0.490798,0.053892,0.366667,0.485981,0.883495,0.604651,0.453704,0.720588,0.440559,1.000000,0.566845,0.805714,0.470297,0.464516,40.4,41.6,66.2,50.1
4,5,Connecticut,BE,25-8,0.859639,6,0.184171,20,0.941291,5,25.818120,8.181880,13,7,13-7,0.664885,0.502880,0.734646,0.668298,0.502880,0.734646,0.716468,0.787490,108.127506,100.573539,108.182491,100.512193,0.811394,0.235673,124.326509,97.443387,0.942776,13,-0.043554,1513,1367,1355.6500,1.116070,1.008372,0,0.650000,0.833576,14,276,0.955744,0.687117,0.239521,0.385185,0.604361,1.000000,0.645349,0.527778,0.514706,0.636364,0.743243,0.695187,0.737143,0.668317,0.374194,42.0,30.3,62.6,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,359,Delaware St.,MEAC,23-Jun,0.124941,356,0.729952,319,0.093425,359,7.000000,22.000000,4,10,10-Apr,0.376478,0.454884,0.289759,0.376478,0.454884,0.289759,0.868691,0.788911,101.915226,105.624696,101.915226,105.624696,0.274860,0.781581,87.839734,93.121099,0.338173,4,-0.053654,938,1055,990.2250,0.947259,1.065414,0,0.285714,0.163445,343,297,0.179334,0.208589,0.772455,0.307407,0.551402,0.558252,0.459302,0.953704,0.485294,0.335664,0.101351,0.251337,0.462857,0.366337,0.625806,31.0,43.2,50.9,62.6
359,360,IUPUI,Horz,27-May,0.243230,330,0.941479,360,0.083555,360,5.000000,27.000000,2,18,18-Feb,0.350852,0.298412,0.376956,0.350852,0.298412,0.376956,0.906959,0.878243,102.312337,107.724320,102.312337,107.724320,0.348339,0.975781,73.093119,104.709037,0.015771,2,-0.080168,1350,1599,1363.1250,0.990371,1.173040,0,0.100000,0.019877,361,334,0.1411

### 

In [72]:
pwr_rats = []
for team in scaled_stat_df['team']:
    pwr_rat = team_power_rating_generator(scaled_stat_df, team, stat_weights)
    pwr_rats.append(pwr_rat)

scaled_stat_df['pwr_rat'] = pwr_rats
scaled_stat_pwrrat_df = scaled_stat_df.sort_values('pwr_rat',ascending=False)
scaled_stat_pwrrat_df[['team','pwr_rat']].to_csv('sim_output/2023/original/team_power_ratings.csv')

In [67]:
team_power_rating_generator(scaled_stat_df, 'North Carolina St.', stat_weights)

52.83041653681788

In [73]:
scaled_stat_pwrrat_df = scaled_stat_df.sort_values('pwr_rat',ascending=False)

In [75]:
scaled_stat_pwrrat_df[['team','pwr_rat']].to_csv('sim_output/2023/original/team_power_ratings.csv')